In [1]:
import numpy as np
import torch

xmeans_path = 'data/Xmeans dataset/'
encode_file = 'cluster data.txt'
#result_file = 'result MinkowskiDistance.txt'
#result_file = 'result EuclideanDistance.txt'
#result_file = 'result ChebyshevDistance.txt'
result_file = 'result ManhattanDistance.txt'



encode = np.loadtxt(xmeans_path + encode_file, delimiter=',')
result = np.loadtxt(xmeans_path + result_file, dtype=np.int32)

In [2]:
result_labels = np.unique(result)
centers = np.zeros((len(result_labels), encode.shape[1]), dtype=np.float64)

In [3]:
for i, label in enumerate(result_labels):
    centers[i]=np.mean(encode[result==label], axis=0)

In [4]:
class Xmeans_Classifier():
    def __init__(self, centers, labels = None, ord=None):
        self.centers = centers
        if labels is None:
            self.labels = [i for i in range(len(centers))]
        else:
            self.labels = labels
        self.ord = ord
        
    def __call__(self, x):
        dist = np.linalg.norm(x-self.centers, self.ord, axis=1)
        return self.labels[np.argmin(dist)]


In [5]:
classifier = Xmeans_Classifier(centers, result_labels, 1)
check_result = np.empty_like(result)
for i, e in enumerate(encode):
    check_result[i] = classifier(e)

In [6]:
np.sum(check_result!=result)

3508

In [7]:
import pandas as pd
from model import BatchProgramCC
from gensim.models.word2vec import Word2Vec

def get_device():
    if torch.cuda.is_available():
        if torch.cuda.get_device_name(0) == 'GeForce GT 730':
            device = 'cpu'
        else:
            device = 'cuda'
    else:
        device = 'cpu'
    return torch.device(device)

device = get_device()

root = 'data/'
train_data = pd.read_pickle(root+'train/blocks.pkl')#.sample(frac=0.1)
validation_data = pd.read_pickle(root + 'dev/blocks.pkl')#.sample(frac=0.1)
test_data = pd.read_pickle(root+'test/blocks.pkl')#.sample(frac=0.1)
ast_embedded = pd.concat([train_data, validation_data, test_data], ignore_index=True)
del train_data
del validation_data
del test_data
ast_embedded['label'] -= 1  # from [1, 104] to [0, 103]


word2vec = Word2Vec.load(root+"train/embedding/node_w2v_128").wv
MAX_TOKENS = word2vec.vectors.shape[0]
EMBEDDING_DIM = word2vec.vectors.shape[1]

HIDDEN_DIM = 100
ENCODE_DIM = 128
LABELS = 104
BATCH_SIZE = 128

model_file = '20201006_151536 model for classification task/20201007_013732 model for classification task epoch_35 val.pt'
save_dir = '---- classification model/'


model = BatchProgramCC(EMBEDDING_DIM,
                       HIDDEN_DIM,
                       MAX_TOKENS+1,
                       ENCODE_DIM,
                       LABELS,
                       BATCH_SIZE,
                       device).to(device)

model.load_state_dict(torch.load(save_dir + model_file)['param'])
model.eval()

C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:102: UserWarning: 
    Found GPU0 GeForce GT 730 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))
C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:125: UserWarning: 
GeForce GT 730 with CUDA capability sm_30 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the GeForce GT 730 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


BatchProgramCC(
  (encoder): BatchTreeEncoder(
    (embedding): Embedding(8189, 128)
    (W_c): Linear(in_features=128, out_features=128, bias=True)
  )
  (bigru): GRU(128, 100, batch_first=True, bidirectional=True)
  (hidden2label): Linear(in_features=200, out_features=104, bias=True)
)

In [22]:
import math
class dataloader():
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.idx = 0
        self.max_idx = len(dataset)
        self.len = math.ceil(len(self.dataset) / self.batch_size)
        
    def __call__(self):
        pass
    
    def __len__(self):
        return self.len
    
    def __iter__(self):
        self.idx = 0
        return self
    
    def __next__(self):
        if self.idx >= self.max_idx:
            raise StopIteration
        tmp = self.dataset.iloc[self.idx: self.idx+self.batch_size]
        x = []
        for _, item in tmp.iterrows():
            x.append(item['code'])
        self.idx += self.batch_size
        return x
    
def count_most(obj):
    if len(obj) > 0:
        return np.max([np.sum(obj==i) for i in set(obj)])  # 出现次数最多的出现了多少次
    else:
        return 0

In [9]:
ast_embedded_loader = dataloader(ast_embedded, BATCH_SIZE)

features = []
with torch.no_grad():
    for x in ast_embedded_loader:
        model.batch_size = len(x)
        model.hidden = model.init_hidden()
        f = model.encode(x).cpu().numpy()
        features.extend(f)
        
#features=np.array(features)

In [10]:
cluster_label = np.array([classifier(f) for f in features])
label = np.array(list(ast_embedded['label']))


In [23]:
count = 0
for i in result_labels:
    count += count_most(label[cluster_label==i])

In [51]:
# EuclideanDistance
count/len(label)

0.9845964500682679

In [12]:
# ChebyshevDistance
count/len(label)

0.9793273206284494

In [24]:
# ManhattanDistance
count/len(label)

0.985327205246053

In [8]:
class vector_calculation():
    def __init__(self, model):
        self.model = model
    def __call__(self, x):
        with torch.no_grad():
            vector = self.model.encode([x]).reshape(-1)
        return vector.cpu().numpy()


In [9]:
ast_embedded['features'] = ast_embedded['code'].apply(vector_calculation(model))

In [10]:
ast_embedded['cluster_label'] = ast_embedded['features'].apply(classifier)

In [11]:
ast_embedded

,id,code,label,features,cluster_label
0,46887,"[[32, [2, [30, [40, [81]]]]], [6], [2, [9, [4]...",92,"[0.36035618, 0.4078555, 0.9507338, 0.103660464...",358.0
1,4970,"[[32, [2, [30, [40, [4]]]]], [6], [2, [13, [4]...",41,"[0.3429833, 0.4664078, -0.02911116, 0.18196319...",372.0
2,50521,"[[32, [2, [30, [1296, [4]]]]], [6], [11, [36],...",76,"[0.23876348, 0.2395441, 0.96246934, 0.1054171,...",96.0
3,38406,"[[32, [2, [30, [40, [4]]]]], [6], [2, [23, [23...",62,"[0.34282124, 0.42330775, 0.9673714, 0.8964845,...",450.0
4,44230,"[[32, [2, [30, [40, [4]]]]], [6], [2, [411, [4...",79,"[0.34282124, 0.4148038, 0.9456779, 0.99406946,...",164.0
...,...,...,...,...,...
51996,43422,"[[32, [2, [30, [40, [4]]]]], [6], [2, [68, [35...",87,"[0.34282124, 0.35292643, 0.9748325, 0.12975913...",561.0
51997,41030,"[[32, [2, [30, [40, [4]]]]], [6], [2, [23, [71...",16,"[0.34282124, 0.26806018, 0.98776746, 0.1297591...",214.0
51998,2878,"[[32, [2, [30, [40, [4]]]]], [6], [2, [13, [4]...",88,"[0.35464072, 0.4389839, 0.9766421, 0.12975913,...",47.0
51999,40365,"[[32, [2, [30, [40, [4]]]]], [6], [2, [23, [9,...",63,"[0.34282124, 0.30945837, 0.97175664, 0.8699453...",239.0
